# Creating files with script mode


## 1. Get data


In [5]:
import os
import zipfile
import requests
from pathlib import Path

# Setting up path to data folder
data_path = Path("../data/")
image_path = data_path / "pizza_steak_sushi"

# Downloading data if it doesn't exist
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get(
        "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
    )
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzipping downloaded file
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Removing zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find ../data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


## 2. Create Datasets and DataLoaders


In [8]:
%%writefile ../modules/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for image classification data.
"""

import os
from pathlib import Path

from torch.utils.data import DataLoader
from torchvision import datasets, transforms


def create_dataloaders(
    train_dir: str | Path,
    test_dir: str | Path,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size: int,
):
    """
    Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns them into PyTorch Datasets
    and then into PyTorch DataLoaders.

    Args;
        train_dir (str | Path): Path to training directory.
        test_dir (str | Path): Path to testing directory.
        train_transform (transforms.Compose): `torchvision` transforms to perform on training data data.
        test_transform (transforms.Compose): `torchvision` transforms to perform on testing data.
        batch_size (int): Number of samples per batch in each of the DataLoaders.

    Returns:
        `Tuple[DataLoader,DataLoader,str]`: A tuple of `(train_dataloader, test_dataloader, class_names)`,
        where `class_names` is a list of the target classes.
    """

    # Using ImageFolder to create datasets
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    # Getting class names
    class_names = train_data.classes

    # Turn images into dataloaders
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

Overwriting ../modules/data_setup.py


In [12]:
%%writefile ../modules/model_builder.py

"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn


class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    The original architecture can be found here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape (int): An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """

    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_shape,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=0,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting ../modules/model_builder.py


In [ ]:
%%writefile ../modules/engine.py

"""
Contains functions for training and testing a PyTorch model.
"""

import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple


def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
) -> Tuple[float, float]:
    """
    Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode then runs through all of the required training steps
    (forward pass, loss calculation, optimizer step).

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): A DataLoader instance for the model to be trained on.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to help minimize the loss function.
        device (str): A target device's name to compute on.

    Returns:
        `Tuple[float,float]`: Tuple of training loss and training accuracy metrics.
    """

    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: str,
) -> Tuple[float, float]:
    """
    Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to eval mode then runs through forward pass on test dataset.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): A DataLoader instance for the model to be tested on.
        loss_fn (torch.nn.Module): A PyTorch loss function to calculate loss on test data.
        device (str): A target device's name to compute on.

    Returns:
        `Tuple[float,float]`: Tuple of testing loss and testing accuracy metrics.
    """

    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module = torch.nn.CrossEntropyLoss(),
    epochs: int = 5,
    device: str = "cpu",
) -> Dict[str, List[float]]:
    """
    Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode then runs through all of the required training steps
    (forward pass, loss calculation, optimizer step).

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        train_dataloader (torch.utils.data.DataLoader): A DataLoader instance for the model to be trained on.
        test_dataloader (torch.utils.data.DataLoader): A DataLoader instance for the model to be tested on.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to help minimize the loss function.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        device (str): A target device's name to compute on.

    Returns:
        `Dict[str,List[float]]`: Dictionary of training and testing loss, as well as training and testing
        accuracy metrics. Each metric has a value in a list for each epoch.

    """

    results = {
        "train_loss": [],
        "test_loss": [],
        "train_acc": [],
        "test_acc": [],
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        print(
            f"""
======================================      
Epoch:                       |   {epoch + 1}   |  
======================================      
Train Loss:                  | {train_loss:.3f} |

Train Accuracy:              | {train_acc:.3f} |            
--------------------------------------
Test Loss:                   | {test_loss:.3f} |

Test Accuracy:               | {test_acc:.3f} |
              """
        )

        results["train_loss"].append(
            train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss
        )
        results["train_acc"].append(
            train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc
        )
        results["test_loss"].append(
            test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss
        )
        results["test_acc"].append(
            test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc
        )

    return results

Overwriting ../modules/engine.py


In [16]:
%%writefile ../modules/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""

import torch
from pathlib import Path


def save_model(model: torch.nn.Module, target_dir: str | Path, model_name: str) -> None:
    """
    Saves a PyTorch model to a target directory.

    Args:
        model (torch.nn.Module): A PyTorch model to save.
        target_dir (str | Path): Directory in which the model should be saved.
        model_name (str): The filename for the saved model. Should include either
        ".pth" or ".pt" as the file extension
    """
    target_dir_path = target_dir
    if not isinstance(target_dir_path, Path):
        target_dir_path = Path(target_dir)
        
    target_dir_path.mkdir(parents=True, exist_ok=True)

    assert model_name.endswith(".pth") or model_name.endswith(
        ".pt"
    ), "model_name should end with '.pt' or '.pth'."
    model_save_path = target_dir_path / model_name

    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

Writing ../modules/utils.py


In [ ]:
%%writefile ../modules/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "../data/pizza_steak_sushi/train"
test_dir = "../data/pizza_steak_sushi/test"

# Setup target device
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "mps" if torch.mps.is_available() else "cpu"

# Creating transforms
data_transform = transforms.Compose(
    [
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor(),
    ]
)

# Creating DataLoaders with helps from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=data_transform,
    test_transform=data_transform,
    batch_size=BATCH_SIZE,
)

# Creating model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names)
).to(device)

# Setting up loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
    device=device,
)

# Saving model with help from utils.py
utils.save_model(model=model, target_dir="models", model_name="script_mode.pth")

Writing ../modules/train.py
